## Fake News Classifier Using LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
df.shape

(20800, 5)

In [6]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [7]:
###Drop Nan Values
df=df.dropna()
# see i can also replace these 1957 tittles with something else but my
# data is of 20k size out of which if 2k entities gone koi fark nhi padega 
# so directly dropping 

In [8]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [9]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [10]:
## Get the Dependent features
y=df['label']

In [11]:
X.shape

(18285, 4)

In [12]:
y.shape

(18285,)

In [13]:
import tensorflow as tf

In [14]:
tf.__version__

'2.14.0'

In [12]:
#all these are already used in wordembedding wali nb so first do that
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [13]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [14]:
messages=X.copy()

In [15]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [32]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [21]:
messages.reset_index(inplace=True)
#just given proper index so that later i apply say any preprossesing 
# tech to these sent proper index access could be done

In [22]:
messages

,level_0,index,id,title,author,text
0,0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...,...
18280,18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [23]:
import nltk
import re
from nltk.corpus import stopwords

In [24]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/priyanshugupta/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [26]:
### Dataset Preprocessing
# we did stopwords and stemming 
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [27]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [28]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [29]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[2187, 3001, 1136, 900, 4862, 2919, 3362, 1975, 692, 2260],
 [4866, 4466, 4457, 1735, 3774, 544, 1171],
 [2103, 3061, 2070, 3843],
 [4195, 4301, 4054, 3413, 533, 1747],
 [1143, 3774, 2409, 223, 306, 4573, 3774, 4035, 2797, 3400],
 [4471,
  864,
  1800,
  1651,
  973,
  3492,
  2382,
  4017,
  2500,
  4895,
  2577,
  142,
  251,
  3802,
  1171],
 [4685, 2060, 464, 4207, 753, 3682, 973, 696, 3288, 419, 2866],
 [3069, 2287, 99, 1918, 230, 4384, 3492, 3109, 3288, 419, 2866],
 [3942, 3140, 4021, 3373, 937, 2550, 3010, 3157, 3492, 2580],
 [3771, 4135, 2258, 2675, 4297, 1354, 3909, 4808],
 [4394, 2042, 3407, 3578, 1170, 2358, 2855, 3788, 2855, 3617, 2690],
 [3413, 4085, 4862, 2550, 3492, 230],
 [3814, 2073, 80, 4325, 1948, 124, 1557, 3446, 3240],
 [175, 286, 3321, 168, 131, 294, 3569, 3288, 419, 2866],
 [3775, 3151, 2149, 55, 2826, 3288, 419, 2866],
 [4010, 2823, 1484, 1838, 2296, 2105, 4139, 951, 2778, 4844],
 [675, 2301, 4466],
 [587, 4908, 270, 1279, 3492, 1974, 210, 1171],
 [4103, 4410, 

In [30]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [31]:
onehot_repr[1]

[4866, 4466, 4457, 1735, 3774, 544, 1171]

### Embedding Representation

In [32]:
#padding done
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[2187 3001 1136 ...    0    0    0]
 [4866 4466 4457 ...    0    0    0]
 [2103 3061 2070 ...    0    0    0]
 ...
 [ 798 2407 4686 ...    0    0    0]
 [1168  230 2674 ...    0    0    0]
 [3752 1320  964 ...    0    0    0]]


In [33]:
embedded_docs[1]

array([4866, 4466, 4457, 1735, 3774,  544, 1171,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [36]:
embedded_docs[0]

array([2187, 3001, 1136,  900, 4862, 2919, 3362, 1975,  692, 2260,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [37]:
#now real embedding part 
## Creating model
# this embedding_features = dimensions
# also here we have done both embedding layer and lstm layer here only
embedding_vector_features=40 ##features representation
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
#lstm with 100 neurons
model.add(LSTM(100))
#binary output so sigmoid fn and binary_crossentropy
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_1 (LSTM)               (None, 100)               56400     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [55]:
# embedded docs was my input vector so showing its shape
len(embedded_docs)

18285

In [58]:
# y is my output  
y.shape

(18285,)

In [59]:
embedded_docs

array([[2187, 3001, 1136, ...,    0,    0,    0],
       [4866, 4466, 4457, ...,    0,    0,    0],
       [2103, 3061, 2070, ...,    0,    0,    0],
       ...,
       [ 798, 2407, 4686, ...,    0,    0,    0],
       [1168,  230, 2674, ...,    0,    0,    0],
       [3752, 1320,  964, ...,    0,    0,    0]], dtype=int32)

In [60]:
y

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 18285, dtype: int64

In [51]:
# see now my model is ready with all its neural layers just put input in array form 
# see embedded_docs was initially array so no need to do it but y was not so for that we need to do it 
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [62]:
y_final

array([1, 0, 1, ..., 0, 1, 1])

In [64]:
X_final

array([[2187, 3001, 1136, ...,    0,    0,    0],
       [4866, 4466, 4457, ...,    0,    0,    0],
       [2103, 3061, 2070, ...,    0,    0,    0],
       ...,
       [ 798, 2407, 4686, ...,    0,    0,    0],
       [1168,  230, 2674, ...,    0,    0,    0],
       [3752, 1320,  964, ...,    0,    0,    0]], dtype=int32)

In [65]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [118]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 3s 14ms/step - loss: 0.3194 - accuracy: 0.8414 - val_loss: 0.2000 - val_accuracy: 0.9168
Epoch 2/10
192/192 [==============================] - 3s 13ms/step - loss: 0.1463 - accuracy: 0.9448 - val_loss: 0.1953 - val_accuracy: 0.9211
Epoch 3/10
192/192 [==============================] - 3s 13ms/step - loss: 0.1054 - accuracy: 0.9611 - val_loss: 0.2168 - val_accuracy: 0.9162
Epoch 4/10
192/192 [==============================] - 3s 14ms/step - loss: 0.0820 - accuracy: 0.9715 - val_loss: 0.2688 - val_accuracy: 0.9155
Epoch 5/10
192/192 [==============================] - 3s 14ms/step - loss: 0.0677 - accuracy: 0.9770 - val_loss: 0.2390 - val_accuracy: 0.9138
Epoch 6/10
192/192 [==============================] - 3s 13ms/step - loss: 0.0564 - accuracy: 0.9812 - val_loss: 0.2827 - val_accuracy: 0.9158
Epoch 7/10
192/192 [==============================] - 3s 14ms/step - loss: 0.0482 - accuracy: 0.9858 - val_loss: 0.3466 - val_accuracy: 0.9112

### Performance Metrics And Accuracy

In [119]:
y_pred=model.predict(X_test)

189/189 [==============================] - 0s 2ms/step


In [120]:
y_pred

array([[9.9857897e-01],
       [2.8495473e-04],
       [1.9079125e-02],
       ...,
       [9.7545638e-04],
       [9.9847752e-01],
       [7.1748775e-01]], dtype=float32)

In [121]:
#just saying where y_pred is>0.6 take it as 1
y_pred=np.where(y_pred > 0.5, 1,0)

In [122]:
from sklearn.metrics import confusion_matrix

In [123]:
confusion_matrix(y_test,y_pred)

array([[3150,  269],
       [ 280, 2336]])

In [124]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9090306545153273

In [125]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.92      0.92      0.92      3419
           1       0.90      0.89      0.89      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

